In [1]:
import os
import h5py
import keras
import numpy as np
from keras.applications.resnet50 import ResNet50 
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.utils.io_utils import HDF5Matrix
%matplotlib notebook
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# extract and process Food101 data
base_path = os.path.join('.', 'food41')
train_h5_path = os.path.join(base_path, 'food_c101_n10099_r32x32x1.h5')
test_h5_path = os.path.join(base_path, 'food_test_c101_n1000_r32x32x1.h5')

X_train = HDF5Matrix(train_h5_path, 'images')
y_train = HDF5Matrix(train_h5_path, 'category')

X_test = HDF5Matrix(test_h5_path, 'images')
y_test = HDF5Matrix(test_h5_path, 'category')

/Users/lle/anaconda3/lib/python3.6/site-packages/keras/utils/io_utils.py:60: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(datapath)


In [3]:
# load pre-trained vgg model
base_model = VGG16(weights= None, include_top=False, 
                      input_shape= (32, 32, 1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(101, activation= 'softmax')(x)
vgg = Model(inputs = base_model.input, outputs = predictions)

In [4]:
# train with vgg
vgg.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
vgg_his = vgg.fit(X_train, y_train, 
                  validation_data=(X_test, y_test), 
                  batch_size = 256,
                  epochs = 5, shuffle="batch")

Train on 10099 samples, validate on 1000 samples
Epoch 1/5
10099/10099 [==============================] - 685s 68ms/step - loss: 4.6158 - accuracy: 0.0082 - val_loss: 4.6152 - val_accuracy: 0.0080
Epoch 2/5
10099/10099 [==============================] - 634s 63ms/step - loss: 4.6144 - accuracy: 0.0100 - val_loss: 4.6162 - val_accuracy: 0.0060
Epoch 3/5
10099/10099 [==============================] - 651s 64ms/step - loss: 4.6137 - accuracy: 0.0109 - val_loss: 4.6174 - val_accuracy: 0.0060
Epoch 4/5
10099/10099 [==============================] - 786s 78ms/step - loss: 4.6132 - accuracy: 0.0115 - val_loss: 4.6175 - val_accuracy: 0.0060
Epoch 5/5
10099/10099 [==============================] - 826s 82ms/step - loss: 4.6129 - accuracy: 0.0099 - val_loss: 4.6183 - val_accuracy: 0.0060
